In [1]:
import numpy as np
import open3d as o3d
import copy
import trimesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## File IO

In [2]:
stl_mesh = o3d.io.read_triangle_mesh("stl_mesh.stl")
obj_mesh = o3d.io.read_triangle_mesh("obj_mesh.obj")

## Visualization

In [3]:
mesh = obj_mesh
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])

In [4]:
# o3d.visualization.draw_geometries([mesh, mesh_frame])

In [5]:
# scaled_mesh = copy.deepcopy(mesh)
# scaled_mesh.scale(1 / np.max(scaled_mesh.get_max_bound() - scaled_mesh.get_min_bound()), center=scaled_mesh.get_center())
# o3d.visualization.draw_geometries([scaled_mesh, mesh_frame])

## Mesh Cleaning

*reference: https://support.shapeways.com/hc/en-us/articles/360007107674-Tips-for-successful-modeling*

In [20]:
mesh

TriangleMesh with 134082 points and 44694 triangles.

In [21]:
o3d.visualization.draw_geometries([mesh])

In [24]:
processed_mesh = copy.deepcopy(mesh)

## Checks

In [25]:
# tests if all vertices are manifold
print(processed_mesh.is_vertex_manifold())
# tests if all edges are manifold
# [allow_boundary_edges=True] non-manifold edges: edges with adjacent triangles > 2 
# [allow_boundary_edges=False] non-manifold edges: edges with adjacent triangles != 2 
print(processed_mesh.is_edge_manifold(allow_boundary_edges=True))
# tests if the mesh is watertight
print(processed_mesh.is_watertight())

True
True
False


## Fix Vertices

In [26]:
# removes vertices that have identical coordinates
processed_mesh.remove_duplicated_vertices()
# removes vertices that are not referenced in any triangle
processed_mesh.remove_unreferenced_vertices()
# returns a list of indices to non-manifold vertices
processed_mesh.get_non_manifold_vertices()

IntVector[]

## Fix Edges

In [27]:
# returns a list of indices of non-manifold edges
# [allow_boundary_edges=True] non-manifold edges: edges with adjacent triangles > 2 
# [allow_boundary_edges=False] non-manifold edges: edges with adjacent triangles != 2 
processed_mesh.get_non_manifold_edges(allow_boundary_edges=True)
# removes all non-manifold edges
# by successively deleting triangles with the smallest surface area 
# adjacent to the non-manifold edge until the number of adjacent 
# triangles to the edge is <= 2
processed_mesh.remove_non_manifold_edges()

TriangleMesh with 22332 points and 44690 triangles.

## Fix Triangles

In [28]:
# removes triangles that reference the same three vertices
processed_mesh.remove_duplicated_triangles()
# removes triangles that reference a single vertex multiple times in a single triangle
# usually happens as a result of removing duplicated vertices
processed_mesh.remove_degenerate_triangles()

TriangleMesh with 22332 points and 44690 triangles.

In [29]:
o3d.visualization.draw_geometries([processed_mesh])

## TriMesh

In [32]:
# mesh = o3d.io.read_triangle_mesh("bunny.obj")
tri_mesh = trimesh.Trimesh(vertices=np.array(mesh.vertices), faces=np.array(mesh.triangles))
print(tri_mesh.is_watertight)
tri_mesh.fill_holes()
tri_mesh.fix_normals()
print(tri_mesh.is_watertight)

False
False
